In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/firevedio1/1108330383-preview.mp4
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.154.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.115.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.59.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.58.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.166.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.142.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.44.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.11.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.19.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.169.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.242.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.29.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.182.png
/kaggle/i

In [2]:
pip install opencv-python-headless numpy scikit-learn tensorflow wurlitzer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow logging

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical

# Your TensorFlow code here


2024-07-10 09:53:55.749265: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-10 09:53:55.749419: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-10 09:53:55.945600: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
training_data=[r'/kaggle/input/fire-dataset/fire_dataset/fire_images',
              r'/kaggle/input/fire-dataset/fire_dataset/non_fire_images']

In [6]:
def load_images(data_dir):
    images = []  # Initialize empty list to store the images
    labels = []  # Initialize empty list to store the labels


In [7]:
def load_images(data_dir):
    images = []  # Initialize empty list to store the images
    labels = []  # Initialize empty list to store the labels

    for i in range(len(training_data)):
        folder = training_data[i]  # Get the folder of the images
        label = i  # Get the label of the images
        for filename in os.listdir(folder):
            try:
                img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)  # Read the image file
                if img is not None:  # Ensure the image is loaded
                    img = cv2.resize(img, (48, 48))  # Resize the image
                    images.append(img)  # Append the image
                    labels.append(label)  # Append the label
                else:
                    print(f"Error loading image: {os.path.join(folder, filename)}: Image not found or corrupted")
            except Exception as e:
                print(f"Error loading image: {os.path.join(folder, filename)}: {str(e)}")

    return np.array(images), np.array(labels)  # Return the images and labels as numpy arrays

images, labels = load_images(training_data)
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)  # Split the data into training and testing sets
x_train = x_train.reshape(x_train.shape[0], 48, 48, 1).astype('float32') / 255  # Reshape the data and normalize
x_test = x_test.reshape(x_test.shape[0], 48, 48, 1).astype('float32') / 255

y_train = to_categorical(y_train)  # Convert the labels to categorical
y_test = to_categorical(y_test)

model = Sequential()  # Initialize the model
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))  # Add the first convolution layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))  # Add the second convolution layer

model.add(MaxPooling2D(pool_size=(2, 2)))  # Add the first pooling layer

model.add(Dropout(0.25))  # Add the first dropout layer
model.add(Flatten())  # Add the flatten layer
model.add(Dense(128, activation='relu'))  # Add the first dense layer
model.add(Dropout(0.5))  # Add the second dropout layer
model.add(Dense(2, activation='softmax'))  # Add the output layer

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # Compile the model
model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(x_test, y_test))  # Fit the model

# Save the model in the recommended .keras format
model.save('fire_detection_model.keras')

Error loading image: /kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.189.png: Image not found or corrupted
Epoch 1/10
25/25 [==============================] - 5s 154ms/step - loss: 0.6136 - accuracy: 0.7268 - val_loss: 0.4970 - val_accuracy: 0.7300
Epoch 2/10
25/25 [==============================] - 3s 139ms/step - loss: 0.3991 - accuracy: 0.8158 - val_loss: 0.3401 - val_accuracy: 0.8650
Epoch 3/10
25/25 [==============================] - 4s 142ms/step - loss: 0.3309 - accuracy: 0.8697 - val_loss: 0.3261 - val_accuracy: 0.8850
Epoch 4/10
25/25 [==============================] - 4s 160ms/step - loss: 0.3232 - accuracy: 0.8584 - val_loss: 0.2811 - val_accuracy: 0.8700
Epoch 5/10
25/25 [==============================] - 3s 140ms/step - loss: 0.2786 - accuracy: 0.8897 - val_loss: 0.2839 - val_accuracy: 0.8800
Epoch 6/10
25/25 [==============================] - 4s 143ms/step - loss: 0.2536 - accuracy: 0.8935 - val_loss: 0.2884 - val_accuracy: 0.8850
Epoch 7/10
25/25 [======

OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO

**<h1>Building a fire detection system using Opencv</h1>**

In [8]:
from keras.models import load_model

<h3>Load the model</h3>

In [9]:
model= load_model('/kaggle/working/fire_detection_model.keras')


In [10]:
def detect_fire(frame, treshold=0.5):
    preprocess_frame=cv2.cvtColor(cv2.resize(frame,(48,48)),cv2.COLOR_BGR2GRAY)
    preprocess_frame=np.expand_dims(preprocess_frame, axis=0)
    preprocess_frame=np.expand_dims(preprocess_frame, axis=-1)
    preprocess_frame=np.preprocess_frame.astype("float32")/255 #Normalizing the pixel values
    
    #Make the predictions
    prediction=model.predict(preprocess_frame)
    if prediction[0][1]>=treshold:
        return True
    else:
        return False
    
    #Opening the vedio file
    cap=cv2.VedioCapture("/kaggle/input/firevedio1")
    #check whether the vedio file has been opened sucessfully
    if  not cap.isOpened():
        print("Error:Vedio file cannot be open")
        exit()
    #Read the frames of the vedio file
    while True:
        ret,frame=cap.read() #Variable ret is for determine whether the vedio file is read
        if not ret:
            break   #if NOT read break the loop
        #Detecting whether there is a fire in the vedio
        if detect_fire(frame):
            cv2.rectangle(frame,(100,100),(frame.shape[1]-100,frame.shape[0]-100),(0,0,255),2)#Adding a rectangle on the fire detected area
            cv2.putText(frame,"FIRE DETECTED",(50,50),cv2.FONT_HERSHEY_DUPLEX,1,(0,0,255),2,cv2.LINE_AA)#Specify the font color and font type etc. of the warning message.
        #Display the frame
        cv2.imshow("Vedio",frame)
        if cv2.waitKey(25) & 0xFF == ord("q"): #press q to quit
             break
        #Release the vedio capture object and close all the windows
        cap.release()
        cv2.destroyAllWindows()

Higher the treshold less sensitive the model should be and vice versa,  Then normalize and and resize the data using grayscale, Next adding the bed dimensions you can copy and paste the same pre processed frame.,   After that we have to normalize the pixel values 
* Making the predictions<br/>
if the predicted probability for a fire class is greater than the
threshold.
So we're going to create a conditional statement if predictions.
So we want to access the first index.
Is greater than or equal to the threshold.
Then we are going to return true.Which is 0.5 in this case.
